In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from pyedflib import highlevel
import pandas as pd

In [ ]:
# Define the main application
root = tk.Tk()
root.title("EDF File Viewer and Processor")
root.geometry("800x600")

In [ ]:
# Define functions
def upload_file():
    file_path = filedialog.askopenfilename(filetypes=[("EDF files", "*.edf")])
    if not file_path:
        return
    try:
        # Process the file
        result = process_edf_file(file_path)
        text_area.delete("1.0", tk.END)
        text_area.insert(tk.END, result)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to process file: {e}")

def process_edf_file(file):
    # Read the EDF file
    signals, signal_headers, header = highlevel.read_edf(file)

    # Find saturation and desaturation channels
    def get_pos():
        index_saturation = 0
        info_saturation = ''
        index_desaturation = 0
        info_desaturation = ''

        for index, item in enumerate(signal_headers):
            if item['label'] == 'Saturation':
                index_saturation = index
                info_saturation = item
            if item['label'] == 'Desaturation':
                index_desaturation = index
                info_desaturation = item   

        return index_saturation, info_saturation, index_desaturation, info_desaturation

    index_saturation, _, index_desaturation, _ = get_pos()
    value_saturation = signals[index_saturation]
    value_desaturation = signals[index_desaturation]

    # Process desaturation values
    new_value_desaturation = []
    for index in range(len(value_desaturation)):
        if index % 10 == 0:
            if value_desaturation[index] != 0.0:
                new_value_desaturation.append(100)
            else:
                new_value_desaturation.append(value_desaturation[index])

    if len(new_value_desaturation) != len(value_saturation):
        raise ValueError("Length mismatch between processed desaturation and saturation signals.")

    # Calculate the area under the curve
    start_point = 0
    end_point = 0
    max_val = 0
    res = 0

    for index, value in enumerate(new_value_desaturation):
        if value != 0.0:
            if new_value_desaturation[index-1] == 0.0:
                start_point = index
                max_val = value_saturation[start_point]
            if new_value_desaturation[index+1] == 0.0:
                end_point = index
            
            if (value_saturation[index+1] > max_val) and ((index+1) != 0.0):
                max_val = value_saturation[index+1]

            if end_point > start_point:
                area_res = 0
                for i in range(start_point, end_point):
                    area_res += (max_val - value_saturation[i])
                res += area_res

    # Calculate final results
    total_duration = len(new_value_desaturation)
    processed_value = res / (120 * (total_duration / 3600))

    # Return results as a string
    result = f"Raw Data (Area): {res}\n" \
             f"Total Duration (s): {total_duration}\n" \
             f"Processed Value: {processed_value}"
    return result

# Widgets
upload_button = tk.Button(root, text="Upload EDF File", command=upload_file)
upload_button.pack(pady=10)

text_area = tk.Text(root, wrap=tk.WORD, height=20, width=100)
text_area.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)


2024-11-25 11:55:05.996 Python[14953:6427117] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-25 11:55:05.996 Python[14953:6427117] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
2024-11-25 11:55:07.584 Python[14953:6427117] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


: 

In [ ]:
# label
label = tk.Label(
    text="Hello, Tkinter",
    fg="white",
    bg="black",
    width=10,
    height=10
)
label.pack()

In [ ]:
# Run the main loop
root.mainloop()

2024-11-25 11:31:28.495 Python[10383:6390854] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-25 11:31:28.495 Python[10383:6390854] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
2024-11-25 11:31:30.272 Python[10383:6390854] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
